**The spin systems (aka frames) in this experiment are defined by w1 and w2, which correspond to a covalently bonded C-H pair.**

---

> Remove any protons but the HN, HD, HE, HZ (all amide protons, bb and sc) from w3 before computing the per-frame intensity ranks and relative intensities and plot the distribution of 
* Intensity rank of CA(i)-HA(i)-HN(i) wrt any CA(i)-HA(i)-HN([0-9]+)
* Intensity rank of CA(i)-HA(i)-HN(i-1) wrt any CA(i)-HA(i)-HN([0-9]+)
* Intensity rank of CA(i)-HA(i)-HN(i+1) wrt any CA(i)-HA(i)-HN([0-9]+)
* Relative intensity of CA(i)-HA(i)-HN(i) wrt any CA(i)-HA(i)-HN([0-9]+)
* Relative intensity of CA(i)-HA(i)-HN(i-1) wrt any CA(i)-HA(i)-HN([0-9]+)
* Relative intensity of CA(i)-HA(i)-HN(i+1) wrt any CA(i)-HA(i)-HN([0-9]+)
* Do all the above but only for Gly, which has characteristic CA-HA shifts and not sidechain protons to diffuse the magentization.

 ---

> **For every amino acid type individually**, remove all HN, HD, HE (all amide protons, bb and sc), all HA and all aromatic protons from w3, and count how many times the most intense peak in the frame belongs to residue i and how many it doesn't.

In [1]:
import pandas as pd

from glob import glob

import warnings
warnings.simplefilter('ignore')

from functions import *

In [2]:
# pdb_ids = ['2K52', '2KD0', '2LTM', '2LF2', '2LTM', '2LX7']
heteronucleus = '13CALI'
pdb_ids = [p.split('/')[-1].split('_')[0] for p in glob(f'data/*_{heteronucleus}.list')]
print(pdb_ids)

['2JT1', '2KRS', '2LX7', '1YEZ', '2K52', '6SVC', '2LF2', '2JRM', '6SOW', '2LTM', '2K53', '2L9R', '2JVD', '2KD0', '2MA6', '2K57']


In [3]:
df = concat_peak_lists(pdb_ids=pdb_ids, heteronucleus=heteronucleus)
print(f'Data aggregated from {df.pdb_id.unique().shape[0]} proteins.\n')
df

Data aggregated from 16 proteins.



,pdb_id,res,noe,X,H,Hnoe,height,noe_res,inter,resnum,noe_resnum,res_diff,atom_type,atom_type_pos
0,2JT1,S2CA,HA,57.564,4.126,4.126,104712,S2CA,False,2,2,0,HA,HA_i
1,2JT1,S2CA,HB2,57.564,4.126,3.982,5424,S2CA,False,2,2,0,HB,HB_i
2,2JT1,S2CA,HB3,57.564,4.126,3.983,5424,S2CA,False,2,2,0,HB,HB_i
3,2JT1,S2CB,HA,63.953,3.982,4.126,9280,S2CB,False,2,2,0,HA,HA_i
4,2JT1,S2CB,HB2,63.953,3.982,3.982,265003,S2CB,False,2,2,0,HB,HB_i
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5580,2K57,E55CG,HA,36.309,2.128,4.130,-2322,E55CG,False,55,55,0,HA,HA_i
5581,2K57,E55CG,HB2,36.309,2.128,1.925,-13574,E55CG,False,55,55,0,HB,HB_i
5582,2K57,E55CG,HB3,36.309,2.128,1.925,-13574,E55CG,False,55,55,0,HB,HB_i
5583,2K57,E55CG,HG2,36.309,2.128,2.207,-175479,E55CG,False,55,55,0,HG,HG_i


Simplifying the NOE contact categories: everything that's more than 1 residue away is now "far"

In [4]:
df.loc[df.res_diff.abs() > 1, "atom_type_pos"] = df.loc[df.res_diff.abs() > 1, "atom_type"] + "_far"

Removing the sign of the crosspeaks: here the phase does not matter.

In [5]:
df.loc[:, 'height'] = df.height.abs()

Make a checkpoint and export the data

In [6]:
df.to_csv(f'data/out/all_peak_lists_{heteronucleus}.csv')

### Filtering

1. We are interested only $H^A$ - $C^A$ frames: remove the side-chain peaks

In [7]:
df = df[df['res'].str.endswith('CA')]
# Optional: to make the column prettier an remove redundancy in the frame's name
#df.loc[:, 'res'] = df.res.str.removesuffix('CA')
df.loc[:, 'noe_res'] = df.res.str.removesuffix('CA')

In [8]:
df[df.pdb_id == np.random.choice(pdb_ids)].sample(5)

,pdb_id,res,noe,X,H,Hnoe,height,noe_res,inter,resnum,noe_resnum,res_diff,atom_type,atom_type_pos
1234,6SVC,E31CA,R32H,56.293,4.284,8.631,31298,E31,True,31,32,-1,H,H_i+1
293,6SVC,W7CA,G6HA3,57.464,5.176,4.008,304,W7,True,7,6,1,HA,HA_i-1
984,6SVC,H23CA,HD2,57.447,4.114,7.130,2418,H23,False,23,23,0,HD,HD_i
953,6SVC,N22CA,I24H,50.690,4.349,8.368,1317,N22,True,22,24,-2,H,H_far
1267,6SVC,R32CA,HG2,54.393,2.707,1.140,4423,R32,False,32,32,0,HG,HG_i


2. Removing the non-amide protons from the NOE contacts
   
$H^A$ -> $C^A$ -> $H^N_{(mostly-bb)}$


In [9]:
aa_sidechain_amide_protons = {
    'R': 'HH',
    'N': 'HD',
    'Q': 'HE',
    'H': 'HD',
    'K': 'HZ',
    'W': 'HE',
}

sc_amide_mask = df.apply(lambda row: is_sc_amide(row, aa_sidechain_amide_protons), axis=1)
# Leave only amides: either in sidechain (according to the mask) or in the backbone (named H)
df_hn = df[sc_amide_mask | (df.atom_type == 'H')]
df_hn.tail(7)

,pdb_id,res,noe,X,H,Hnoe,height,noe_res,inter,resnum,noe_resnum,res_diff,atom_type,atom_type_pos
5307,2K57,L53CA,L54H,56.583,4.120,8.377,10562,L53,True,53,54,-1,H,H_i+1
5312,2K57,L53CA,E55H,56.583,4.120,8.097,3283,L53,True,53,55,-2,H,H_far
5451,2K57,L54CA,L53H,55.340,4.281,8.159,3775,L54,True,54,53,1,H,H_i-1
5456,2K57,L54CA,H,55.340,4.281,8.377,6644,L54,False,54,54,0,H,H_i
5463,2K57,L54CA,E55H,55.340,4.281,8.097,9623,L54,True,54,55,-1,H,H_i+1
5550,2K57,E55CA,L54H,56.905,4.130,8.377,7557,E55,True,55,54,1,H,H_i-1
5554,2K57,E55CA,H,56.905,4.130,8.097,3120,E55,False,55,55,0,H,H_i


There are very few sidechain amide NOEs. That's expected: they are usually too far away from HA for any amino acid type

In [10]:
df_hn.atom_type.unique()

array(['H', 'HE', 'HD', 'HZ'], dtype=object)

### Processing the intensities

In [11]:
print(f'Removing {df_hn.duplicated().sum()} duplicated rows')
df_hn = df_hn.drop_duplicates(['resnum', 'noe_resnum', 'atom_type_pos'])

Removing 18 duplicated rows


Calculating the intensities relative to the maximum of the frame

In [12]:
df_hn.insert(7, 'rel_height', df_hn['height'].to_frame() / df_hn[['pdb_id', 'res', 'height']].groupby(['pdb_id', 'res']).transform('max'))

Calculting atom ranks

Duplicated data points occur when there is entry for both GlyHA2 and HA3 with the same chemical shift (which is the most comon sceario, because those protons are rarily distingushable physically)

In [13]:
df_hn['rank'] = df_hn[['pdb_id', 'res', 'rel_height']].groupby(['pdb_id', 'res'], as_index=False)["rel_height"]\
                        .rank(method='min', ascending=False)\
                        .astype('category')

In [14]:
df_hn.tail(7)

,pdb_id,res,noe,X,H,Hnoe,height,rel_height,noe_res,inter,resnum,noe_resnum,res_diff,atom_type,atom_type_pos,rank
4978,2K57,V50CA,V6H,62.302,4.306,9.186,611,0.795573,V50,True,50,6,44,H,H_far,2.0
4980,2K57,V50CA,I7H,62.302,4.306,9.302,192,0.250000,V50,True,50,7,43,H,H_far,3.0
4986,2K57,V50CA,T8H,62.302,4.306,8.986,768,1.000000,V50,True,50,8,42,H,H_far,1.0
5093,2K57,K51CA,V6H,54.703,4.738,9.186,1133,1.000000,K51,True,51,6,45,H,H_far,1.0
5234,2K57,D52CA,T5H,54.865,4.531,8.235,829,0.923163,D52,True,52,5,47,H,H_far,2.0
5238,2K57,D52CA,V6H,54.865,4.531,9.186,898,1.000000,D52,True,52,6,46,H,H_far,1.0
5292,2K57,L53CA,V6H,56.583,4.120,9.186,330,1.000000,L53,True,53,6,47,H,H_far,1.0


In [15]:
# Checking how the data looks
df_hn.loc[df_hn.res == "V57CA"]

,pdb_id,res,noe,X,H,Hnoe,height,rel_height,noe_res,inter,resnum,noe_resnum,res_diff,atom_type,atom_type_pos,rank
3349,2JT1,V57CA,H,62.834,3.820,8.279,778,0.295144,V57,False,57,57,0,H,H_i,2.0
3354,2JT1,V57CA,N58H,62.834,3.820,8.383,2636,1.000000,V57,True,57,58,-1,H,H_i+1,1.0
3008,2LX7,V57CA,R7H,58.700,5.190,8.790,2738,1.000000,V57,True,57,7,50,H,H_far,1.0
3011,2LX7,V57CA,L9H,58.700,5.190,9.340,1715,0.626370,V57,True,57,9,48,H,H_far,2.0
3090,1YEZ,V57CA,F63H,63.728,4.303,9.112,26,1.000000,V57,True,57,63,-6,H,H_far,1.0


Making another checkpoint

In [16]:
df_hn.to_csv(f'data/out/noe_to_HN_rel_int_{heteronucleus}.csv')

### Leaving only Gly as residues $i$ (i.e. in w1 and w2 dimensions)

$H^A_{Gly} -> C^A_{Gly} - > H^{amide}$

In [17]:
df_gly = df_hn[df_hn.res.str.match(r'^G')]
df_gly.duplicated().sum()

0

In [18]:
print(f'We are dealing with the total of {(df_gly.pdb_id + df_gly.res).unique().shape[0]} frames')

We are dealing with the total of 50 frames


**How many NOEs does each Gly frame has?**

In [19]:
df_gly[["pdb_id", "res"]].value_counts() #.sort_index()

pdb_id  res   
2KRS    G3CA      8
2LF2    G146CA    8
        G150CA    7
        G63CA     6
2JRM    G16CA     6
2LF2    G94CA     6
2KRS    G52CA     5
2LF2    G97CA     5
2JRM    G20CA     4
2LX7    G42CA     4
2K57    G28CA     4
2LF2    G122CA    4
        G103CA    3
1YEZ    G30CA     3
2LF2    G165CA    3
2JT1    G52CA     3
        G35CA     3
2KRS    G16CA     2
2LF2    G71CA     2
2LTM    G67CA     2
2KRS    G49CA     2
        G41CA     2
2LX7    G50CA     2
2KRS    G25CA     2
        G18CA     2
2JT1    G60CA     2
2JRM    G23CA     2
2KD0    G73CA     2
2JRM    G43CA     2
2K52    G31CA     2
        G20CA     2
        G11CA     2
2JT1    G22CA     2
        G65CA     2
        G62CA     2
1YEZ    G47CA     1
2LX7    G47CA     1
2JRM    G42CA     1
2LX7    G26CA     1
2LTM    G60CA     1
2LF2    G62CA     1
        G74CA     1
2KRS    G22CA     1
2K52    G49CA     1
2K57    G12CA     1
        G37CA     1
2KD0    G51CA     1
        G58CA     1
        G75CA     1
2MA6 

In [20]:
#df_gly[df_gly.pdb_id == np.random.choice(pdb_ids)]
df_gly[df_gly.pdb_id == "2KRS"]

,pdb_id,res,noe,X,H,Hnoe,height,rel_height,noe_res,inter,resnum,noe_resnum,res_diff,atom_type,atom_type_pos,rank
132,2KRS,G3CA,Q2H,44.3,3.07,9.33,72,0.015408,G3,True,3,2,1,H,H_i-1,8.0
136,2KRS,G3CA,H,44.3,3.07,8.86,4673,1.000000,G3,False,3,3,0,H,H_i,1.0
144,2KRS,G3CA,V33H,44.3,3.07,8.65,386,0.082602,G3,True,3,33,-30,H,H_far,4.0
150,2KRS,G3CA,T60H,44.3,3.07,8.98,1587,0.339611,G3,True,3,60,-57,H,H_far,2.0
151,2KRS,G3CA,I61H,44.3,3.07,8.68,189,0.040445,G3,True,3,61,-58,H,H_far,5.0
154,2KRS,G3CA,V62H,44.3,3.07,8.37,1082,0.231543,G3,True,3,62,-59,H,H_far,3.0
167,2KRS,G3CA,V5H,44.3,5.25,8.75,179,0.038305,G3,True,3,5,-2,H,H_far,7.0
172,2KRS,G3CA,I35H,44.3,5.25,8.61,180,0.038519,G3,True,3,35,-32,H,H_far,6.0
1334,2KRS,G16CA,S15H,43.7,1.05,9.01,32,1.000000,G16,True,16,15,1,H,H_i-1,1.0
1348,2KRS,G16CA,Y53H,43.7,1.05,9.26,1,0.031250,G16,True,16,53,-37,H,H_far,2.0


In [21]:
df_gly.to_csv(f'data/out/gly_noe_to_HN_{heteronucleus}.csv')